In [41]:
import socket
import pickle
import pandas as pd
import numpy as np
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

In [42]:
dataset_path = 'heart2ex.csv'
data = pd.read_csv(dataset_path)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [43]:
def build_model():
    model = Sequential([
        Dense(16, activation='relu', input_shape=(13,)),
        Dense(8, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=SGD(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [44]:
server_ip ='localhost'
server_port = 5000
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect((server_ip, server_port))

In [45]:
client_private_key = ec.generate_private_key(ec.SECP256R1())
client_public_key = client_private_key.public_key()


In [46]:
server_public_key_pem = client_socket.recv(1024)
server_public_key = serialization.load_pem_public_key(server_public_key_pem)

In [47]:
client_socket.send(client_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
))

178

In [48]:
num_rounds = 200
for round_num in range(num_rounds):
    print(f"Starting round {round_num + 1}/{num_rounds}")
    
  
    encrypted_data = client_socket.recv(4096)
    shared_key = client_private_key.exchange(ec.ECDH(), server_public_key)
    derived_key = HKDF(algorithm=hashes.SHA256(), length=32, salt=None, info=b'handshake data').derive(shared_key)
    aesgcm = AESGCM(derived_key)
    nonce = b'\x00' * 12  
    data = aesgcm.decrypt(nonce, encrypted_data, None)
    global_weights = pickle.loads(data)
    

    model = build_model()
    if global_weights is not None:
        model.set_weights(global_weights)

    model.fit(X, y, epochs=1, verbose=0)

    local_weights = model.get_weights()
    data = pickle.dumps(local_weights)
    encrypted_data = aesgcm.encrypt(nonce, data, None)
    
 
    client_socket.send(encrypted_data)
    
    print(f"Round {round_num + 1} complete.")


client_socket.close()

print("Federated learning complete and connection closed.")


Starting round 1/200


c:\Users\krish\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Round 1 complete.
Starting round 2/200
Round 2 complete.
Starting round 3/200
Round 3 complete.
Starting round 4/200
Round 4 complete.
Starting round 5/200
Round 5 complete.
Starting round 6/200
Round 6 complete.
Starting round 7/200
Round 7 complete.
Starting round 8/200
Round 8 complete.
Starting round 9/200
Round 9 complete.
Starting round 10/200
Round 10 complete.
Starting round 11/200
Round 11 complete.
Starting round 12/200
Round 12 complete.
Starting round 13/200
Round 13 complete.
Starting round 14/200
Round 14 complete.
Starting round 15/200
Round 15 complete.
Starting round 16/200
Round 16 complete.
Starting round 17/200
Round 17 complete.
Starting round 18/200
Round 18 complete.
Starting round 19/200
Round 19 complete.
Starting round 20/200
Round 20 complete.
Starting round 21/200
Round 21 complete.
Starting round 22/200
Round 22 complete.
Starting round 23/200
Round 23 complete.
Starting round 24/200
Round 24 complete.
Starting round 25/200
Round 25 complete.
Starting round